In [1]:
import pandas as pd

import nltk
import spacy
from nltk.corpus import stopwords

from collections import Counter

In [76]:
df2021= pd.read_csv('data/zeus_download_doc_digitalizzati20210514.csv', sep="|", encoding='latin')
df2021 = df2021.map(lambda x: x.lower() if isinstance(x, str) else x)
df2021.loc[df2021['famiglia'] == 'https://www.epc.bnl.it/bnl/filedownload.ashx?i=1&codicefase=5246330&num_prat=559377', 'famiglia'] = 'altro'
df2021.loc[df2021['famiglia'] == 'documenti non classificati', 'famiglia'] = 'altro'
df2021 = df2021[~df2021['nome_epc'].isna()]

df2021['original'] = df2021['nome_epc']
# Keep only alphabetic characters and spaces
df2021['nome_epc'] = df2021['original'].str.replace(r'\/[ie]', '', regex=True)
df2021['nome_epc'] = df2021['nome_epc'].apply(lambda x: ''.join([c for c in x if c.isalpha() or c.isspace()]))
df2021['nome_epc'] = df2021['nome_epc'].str.replace(r'\s{2,}', ' ', regex=True)

# Trim leading and trailing whitespaces
df2021 = df2021.map(lambda x: x.strip() if isinstance(x, str) else x)

In [77]:
altri = df2021[df2021['famiglia'] == 'altro']
noti = df2021[df2021['famiglia'] != 'altro']

In [78]:
filtered_df = df2021[['famiglia', 'nome_epc', 'original']]

In [79]:
counts_dict = {}

for famiglia_value in filtered_df['famiglia'].unique():
    
    # Filter DataFrame for the current 'famiglia'
    filtered = filtered_df[filtered_df['famiglia'] == famiglia_value]
    
    # Count occurrences of each 'nome_epc' for the current 'famiglia'
    nome_epc_counts = filtered['nome_epc'].value_counts().to_dict()
    
    # Add counts to the dictionary
    counts_dict[famiglia_value] = nome_epc_counts

counts_df = pd.DataFrame(counts_dict)
counts_df

,fatture,altro,perizie,atti giudiziali,visure,certificazioni contabili,comunicazioni,imposte,pubblicita',passaggi di status e relativa documentazione,proposte,garanzie e documentazione relativa,contratti e documentazione relativa,closing,delibere
cpnotule,164480.0,3319.0,38.0,1307.0,4.0,2.0,343.0,288.0,161.0,NaN,13.0,3.0,1.0,1.0,NaN
legnotule,69412.0,1537.0,1.0,34.0,NaN,NaN,10.0,27.0,NaN,NaN,5.0,1.0,NaN,NaN,NaN
documento note sulla procedura,2744.0,61961.0,971.0,16328.0,306.0,1263.0,6869.0,886.0,621.0,12.0,1120.0,69.0,143.0,5.0,NaN
documentogenerico,664.0,43451.0,1964.0,5514.0,5896.0,14351.0,11324.0,66.0,280.0,22037.0,1241.0,4524.0,9869.0,7727.0,12.0
messaggio generico,139.0,1434.0,8.0,320.0,14.0,86.0,303.0,55.0,9.0,NaN,24.0,5.0,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ordinario ordinario fideiussioni variazioni condizioni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
inserimento documento mutuo nota discrizione ipoteca abbey national bank surrogato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
estratto notarile prestito e contratto estratto notarile c e con contratto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
atto compravendita rep notaio mercurio paolo dragonetti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [80]:
counts_df = counts_df.drop(columns='altro')
counts_df

,fatture,perizie,atti giudiziali,visure,certificazioni contabili,comunicazioni,imposte,pubblicita',passaggi di status e relativa documentazione,proposte,garanzie e documentazione relativa,contratti e documentazione relativa,closing,delibere
cpnotule,164480.0,38.0,1307.0,4.0,2.0,343.0,288.0,161.0,NaN,13.0,3.0,1.0,1.0,NaN
legnotule,69412.0,1.0,34.0,NaN,NaN,10.0,27.0,NaN,NaN,5.0,1.0,NaN,NaN,NaN
documento note sulla procedura,2744.0,971.0,16328.0,306.0,1263.0,6869.0,886.0,621.0,12.0,1120.0,69.0,143.0,5.0,NaN
documentogenerico,664.0,1964.0,5514.0,5896.0,14351.0,11324.0,66.0,280.0,22037.0,1241.0,4524.0,9869.0,7727.0,12.0
messaggio generico,139.0,8.0,320.0,14.0,86.0,303.0,55.0,9.0,NaN,24.0,5.0,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ordinario ordinario fideiussioni variazioni condizioni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
inserimento documento mutuo nota discrizione ipoteca abbey national bank surrogato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
estratto notarile prestito e contratto estratto notarile c e con contratto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
atto compravendita rep notaio mercurio paolo dragonetti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [81]:
max_family_for_row = counts_df.idxmax(axis=1)

# Create a new DataFrame with 'word' as index and 'famiglia' as the corresponding column
result_df = pd.DataFrame({'nome_epc': max_family_for_row.index, 'famiglia': max_family_for_row.values})

result_df

C:\Users\sophia.dellarciprete\AppData\Local\Temp\ipykernel_23940\2798044695.py:1: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_family_for_row = counts_df.idxmax(axis=1)


,nome_epc,famiglia
0,cpnotule,fatture
1,legnotule,fatture
2,documento note sulla procedura,atti giudiziali
3,documentogenerico,passaggi di status e relativa documentazione
4,messaggio generico,atti giudiziali
...,...,...
22800,ordinario ordinario fideiussioni variazioni co...,contratti e documentazione relativa
22801,inserimento documento mutuo nota discrizione i...,contratti e documentazione relativa
22802,estratto notarile prestito e contratto estratt...,contratti e documentazione relativa
22803,atto compravendita rep notaio mercurio paolo d...,contratti e documentazione relativa


In [82]:
result_df_garanzia = result_df[result_df['famiglia'] == "garanzie e documentazione relativa"]

In [83]:
counts_df_garanzie = counts_df.copy()

In [84]:
counts_df_garanzie = counts_df_garanzie[~counts_df_garanzie.index.isin(result_df_garanzia['nome_epc'])]

In [85]:
counts_df_garanzie = counts_df_garanzie[~counts_df_garanzie['garanzie e documentazione relativa'].isna()]

In [86]:
counts_df_garanzie = counts_df_garanzie.sort_values(by='garanzie e documentazione relativa', ascending=False)
counts_df_garanzie 

,fatture,perizie,atti giudiziali,visure,certificazioni contabili,comunicazioni,imposte,pubblicita',passaggi di status e relativa documentazione,proposte,garanzie e documentazione relativa,contratti e documentazione relativa,closing,delibere
documentogenerico,664.0,1964.0,5514.0,5896.0,14351.0,11324.0,66.0,280.0,22037.0,1241.0,4524.0,9869.0,7727.0,12.0
checklist,NaN,26.0,2.0,16.0,NaN,21.0,NaN,NaN,3670.0,2.0,1176.0,3262.0,NaN,1.0
esame titolo esecutivo di pagamento,NaN,NaN,191.0,14.0,10.0,2.0,NaN,NaN,NaN,1.0,108.0,686.0,NaN,NaN
documento note sulla procedura,2744.0,971.0,16328.0,306.0,1263.0,6869.0,886.0,621.0,12.0,1120.0,69.0,143.0,5.0,NaN
propostedidelibera,5.0,462.0,278.0,1765.0,54.0,704.0,1.0,NaN,16.0,4143.0,30.0,15.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
il legale esterno ha esaminato certificato notarile sostitutivo docc ipocatastale,NaN,NaN,31.0,14.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
il legale esterno ha depositato in cancelleria certificato notarile sostitutivo docc ipocatastale ex art ii comma cpc,NaN,1.0,59.0,14.0,1.0,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
richiesta visura,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
il legale esterno ha esaminato il provvedimento di sospensione della procedura esecutiva,NaN,NaN,354.0,NaN,1.0,6.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [87]:
# Define a function to keep the value of 'garanzie' column and replace other values with '-'
def keep_garanzie(row):
    row = row.apply(lambda x: '-' if x!=row.max() else x)  # Replace other values with '-'
    return row
 
# Apply the function to each row of the DataFrame
modified_df = counts_df_garanzie.drop('garanzie e documentazione relativa', axis=1).apply(keep_garanzie, axis=1)
modified_df = modified_df.merge(counts_df_garanzie['garanzie e documentazione relativa'], left_index=True, right_index=True)
 
modified_df = modified_df.drop(columns = [col for col in modified_df.columns if (modified_df[col] == '-').all()])
modified_df

,fatture,perizie,atti giudiziali,visure,certificazioni contabili,comunicazioni,pubblicita',passaggi di status e relativa documentazione,proposte,contratti e documentazione relativa,garanzie e documentazione relativa
documentogenerico,-,-,-,-,-,-,-,22037.0,-,-,4524.0
checklist,-,-,-,-,-,-,-,3670.0,-,-,1176.0
esame titolo esecutivo di pagamento,-,-,-,-,-,-,-,-,-,686.0,108.0
documento note sulla procedura,-,-,16328.0,-,-,-,-,-,-,-,69.0
propostedidelibera,-,-,-,-,-,-,-,-,4143.0,-,30.0
...,...,...,...,...,...,...,...,...,...,...,...
il legale esterno ha esaminato certificato notarile sostitutivo docc ipocatastale,-,-,31.0,-,-,-,-,-,-,-,1.0
il legale esterno ha depositato in cancelleria certificato notarile sostitutivo docc ipocatastale ex art ii comma cpc,-,-,59.0,-,-,-,-,-,-,-,1.0
richiesta visura,-,-,-,2.0,-,-,-,-,-,-,1.0
il legale esterno ha esaminato il provvedimento di sospensione della procedura esecutiva,-,-,354.0,-,-,-,-,-,-,-,1.0


In [88]:
result_df_contratti = result_df[result_df['famiglia'] == "contratti e documentazione relativa"]
counts_df_contratti = counts_df.copy()
counts_df_contratti = counts_df_contratti[~counts_df_contratti.index.isin(result_df_garanzia['nome_epc'])]
counts_df_contratti = counts_df_contratti[~counts_df_contratti['contratti e documentazione relativa'].isna()]
counts_df_contratti = counts_df_contratti.sort_values(by='contratti e documentazione relativa', ascending=False)
def keep_garanzie(row):
    row = row.apply(lambda x: '-' if x!=row.max() else x)  # Replace other values with '-'
    return row
 
# Apply the function to each row of the DataFrame
modified_df_c = counts_df_contratti.drop('contratti e documentazione relativa', axis=1).apply(keep_garanzie, axis=1)
modified_df_c = modified_df_c.merge(counts_df_contratti['contratti e documentazione relativa'], left_index=True, right_index=True)
 
modified_df_c = modified_df_c.drop(columns = [col for col in modified_df_c.columns if (modified_df_c[col] == '-').all()])
modified_df_c

,fatture,perizie,atti giudiziali,visure,certificazioni contabili,comunicazioni,passaggi di status e relativa documentazione,proposte,garanzie e documentazione relativa,contratti e documentazione relativa
documentogenerico,-,-,-,-,-,-,22037.0,-,-,9869.0
checklist,-,-,-,-,-,-,3670.0,-,-,3262.0
fondiario cf fondiario sif contratto notarilescrittura privata,-,-,-,-,-,-,-,-,-,916.0
esame titolo esecutivo di pagamento,-,-,191.0,-,-,-,-,-,-,686.0
ordinario ordinario tpco contratto cc mod o o mod o a o prestito personale carta di credito,-,-,-,-,-,-,-,-,-,384.0
...,...,...,...,...,...,...,...,...,...,...
richiesta on line informazioni certificati a societï di servizi,-,-,-,-,-,-,-,-,-,1.0
due diligence negativa,-,-,-,-,-,-,-,-,-,1.0
richiesta copia autenticache del decreto di autorizzazione al rilascio di ulteriore copia esecutiva del titolo,-,-,5.0,-,-,-,-,-,-,1.0
fascicolo trovato,-,-,-,-,1.0,-,-,-,-,1.0


In [89]:
contratti = filtered_df[filtered_df['famiglia'] == 'contratti e documentazione relativa']
contratti

,famiglia,nome_epc,original
768,contratti e documentazione relativa,checklist,checklist_1_8816135_
1256,contratti e documentazione relativa,documentogenerico,documentogenerico___
2267,contratti e documentazione relativa,documentogenerico,documentogenerico___
4652,contratti e documentazione relativa,documentogenerico,documentogenerico___
4731,contratti e documentazione relativa,documentogenerico,documentogenerico___
...,...,...,...
1299401,contratti e documentazione relativa,ordinario ordinario tpco contratto cc mod o o ...,ordinario ( 6043000000022058 - ordinari...
1299422,contratti e documentazione relativa,documentogenerico,documentogenerico___
1299432,contratti e documentazione relativa,ordinario ordinario tpco contratto cc mod o o ...,ordinario ( 6483000000001233 - ordinari...
1299510,contratti e documentazione relativa,documentogenerico,documentogenerico___


In [105]:
nltk.download('stopwords')
nlp = spacy.load('it_core_news_sm')
stop_words = set(nlp.Defaults.stop_words)
stop_words.add("o")
stop_words.remove("titolo")

In [91]:
text_data = ' '.join(contratti['nome_epc'].astype(str))

# Process the text with SpaCy
doc = nlp(text_data)
filtered_tokens = [token.text.lower() for token in doc if token.is_alpha and token.text.lower() not in stop_words]

# Calculate word frequencies
word_freq = Counter(filtered_tokens)

# Print the most common words
contratti_mostfreq = pd.DataFrame(list(word_freq.items()), columns=['parola', 'frequenza'])
contratti_mostfreq = contratti_mostfreq.sort_values(by='frequenza', ascending=False)
contratti_mostfreq.head(20)

,parola,frequenza
1,documentogenerico,9869
0,checklist,3262
10,fondiario,2160
13,ordinario,1930
7,contratto,1693
12,sif,1394
8,notarilescrittura,1246
9,privata,1246
16,mod,1216
11,cf,1072


In [92]:
garanzie = filtered_df[filtered_df['famiglia'] == 'garanzie e documentazione relativa']
garanzie

,famiglia,nome_epc,original
581,garanzie e documentazione relativa,documento note sulla procedura,documento / note sulla procedura
2691,garanzie e documentazione relativa,documentogenerico,documentogenerico___
3109,garanzie e documentazione relativa,il legale esterno ha partecipato allesperiment...,il legale esterno ha partecipato all'esperimen...
3895,garanzie e documentazione relativa,documentogenerico,documentogenerico___
5204,garanzie e documentazione relativa,documentogenerico,documentogenerico___
...,...,...,...
1299490,garanzie e documentazione relativa,ipoteca valida non ancora escussa ipoteca cons...,ipoteca ( 01 00007962706 (valida...
1299492,garanzie e documentazione relativa,ipoteca valida non ancora escussa ipoteca cons...,ipoteca ( 01 00007717454 (valida...
1299494,garanzie e documentazione relativa,ipoteca valida non ancora escussa ipoteca conv...,ipoteca ( 01 43000272238 (valida...
1299500,garanzie e documentazione relativa,documentogenerico,documentogenerico___


In [100]:
garanzie[garanzie['nome_epc'].str.contains("titolo", case=False)]

,famiglia,nome_epc,original
57251,garanzie e documentazione relativa,esame titolo esecutivo di pagamento,esame titolo/i esecutivo/i di pagamento
126664,garanzie e documentazione relativa,esame titolo esecutivo di pagamento,esame titolo/i esecutivo/i di pagamento
126919,garanzie e documentazione relativa,il legale esterno ha esaminato il titolo esecu...,il legale esterno ha esaminato il/i titolo/i e...
129507,garanzie e documentazione relativa,il legale esterno ha esaminato il titolo esecu...,il legale esterno ha esaminato il/i titolo/i e...
137834,garanzie e documentazione relativa,esame titolo esecutivo di pagamento,esame titolo/i esecutivo/i di pagamento
...,...,...,...
1276153,garanzie e documentazione relativa,esame titolo esecutivo di pagamento,esame titolo/i esecutivo/i di pagamento
1279367,garanzie e documentazione relativa,esame titolo esecutivo di pagamento,esame titolo/i esecutivo/i di pagamento
1293475,garanzie e documentazione relativa,esame titolo esecutivo di pagamento,esame titolo/i esecutivo/i di pagamento
1296369,garanzie e documentazione relativa,esame titolo esecutivo di pagamento,esame titolo/i esecutivo/i di pagamento


In [106]:
text_data = ' '.join(garanzie['nome_epc'].astype(str))

# Process the text with SpaCy
doc = nlp(text_data)

# Filter out stopwords and non-alphabetic tokens
filtered_tokens = [token.text.lower() for token in doc if token.is_alpha and token.text.lower() not in stop_words]

# Calculate word frequencies
word_freq = Counter(filtered_tokens)

# Print the most common words
garanzie_mostfreq = pd.DataFrame(word_freq.most_common(25))
garanzie_mostfreq.columns = ['parola', 'frequenza']
garanzie_mostfreq

,parola,frequenza
0,ipoteca,7610
1,documentogenerico,4524
2,valida,2734
3,escussa,2734
4,consensuale,1670
5,checklist,1176
6,nota,955
7,ipotecaria,899
8,conv,890
9,cons,863
